In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import import_ipynb
from CandidateExtraction import *
import os
from keras.models import load_model
import json
from Tesseract import *
from keras.models import load_model
from sklearn.decomposition import PCA

importing Jupyter notebook from CandidateExtraction.ipynb
importing Jupyter notebook from Tesseract.ipynb


2023-07-12 15:19:16.351976: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
X = [[1, 0.22514851485148502, 0.04314285714285714],
 [1, 0.19326732673267322, 0.014142857142857138],
 [6, 0.014653465346534555, 0.016142857142857146],
 [1, 0.18772277227722767, 0.027428571428571427],
 [1, 0.7327450980392156, 0.01833333333333334],
 [1, 0.2329411764705882, 0.01878787878787877],
 [6, 0.016274509803921533, 0.01878787878787877],
 [1, 0.08215686274509804, 0.03742424242424244],
 [1, 0.5005882352941176, 0.06242424242424244],
 [1, 0.0007843137254901489, 0.06287878787878787],
 [1, 0.5005882352941176, 0.06242424242424244],
 [1, 0.0007843137254901489, 0.06287878787878787],
 [1, 0.7335294117647059, 0.06242424242424244],
 [1, 0.23372549019607847, 0.06287878787878787],
 [6, 0.017058823529411793, 0.06287878787878787],
 [5, 0.23078431372549024, 0.016666666666666663],
 [6, 0.017058823529411793, 0.018636363636363645],
 [1, 0.07901960784313727, 0.08833333333333335],
 [6, 0.05039215686274512, 0.06878787878787879],
 [6, 0.05039215686274512, 0.06878787878787879],
 [4, 0.10999999999999999, 0.0],
 [1, 0.7327450980392156, 0.01833333333333334],
 [1, 0.2329411764705882, 0.01878787878787877],
 [6, 0.016274509803921533, 0.01878787878787877],
 [6, 0.015490196078431495, 0.02469696969696969],
 [1, 0.22431372549019601, 0.03954545454545455],
 [1, 0.6805882352941176, 0.01712121212121212],
 [1, 0.121764705882353, 0.016969696969696968],
 [1, 0.121764705882353, 0.0001515151515151525],
 [6, 0.07862745098039214, 0.0001515151515151525],
 [1, 0.24274509803921565, 0.03424242424242424],
 [1, 0.22431372549019601, 0.09772727272727272],
 [1, 0.6805882352941176, 0.07530303030303029],
 [1, 0.121764705882353, 0.07515151515151514],
 [1, 0.121764705882353, 0.05833333333333332],
 [6, 0.07862745098039214, 0.05833333333333332],
 [2, 0.09098039215686271, 0.04378787878787879],
 [1, 0.18000000000000005, 0.06954545454545452],
 [1, 0.6362745098039216, 0.04712121212121209],
 [1, 0.07745098039215703, 0.04696969696969694],
 [1, 0.07745098039215703, 0.030151515151515124],
 [6, 0.03431372549019618, 0.030151515151515124],
 [2, 0.046666666666666745, 0.015606060606060596],
 [1, 0.5447058823529414, 0.05545454545454545],
 [1, 0.6682352941176471, 0.017575757575757578],
 [1, 0.11568627450980395, 0.017575757575757578],
 [6, 0.07039215686274525, 0.00015151515151512474],
 [1, 0.5447058823529414, 0.0728787878787879],
 [1, 0.6682352941176471, 0.03500000000000003],
 [1, 0.11568627450980395, 0.03500000000000003],
 [6, 0.07039215686274525, 0.017575757575757578],
 [4, 0.11411764705882366, 0.00015151515151518025],
 [6, 0.07039215686274525, 0.0003030303030303605],
 [3, 0.6992156862745098, 0.030303030303030276],
 [4, 0.2194117647058823, 0.0060606060606061],
 [3, 0.26294117647058823, 0.045151515151515165],
 [1, 0.24588235294117655, 0.047659817351598174],
 [1, 0.23098039215686272, 0.019406392694063926],
 [6, 0.013725490196078494, 0.017123287671232876],
 [1, 0.24588235294117655, 0.047659817351598174],
 [1, 0.23098039215686272, 0.019406392694063926],
 [6, 0.013725490196078494, 0.017123287671232876],
 [1, 0.07882352941176463, 0.07619863013698629],
 [5, 0.076078431372549, 0.045947488584474866],
 [1, 0.25294117647058834, 0.06050228310502284],
 [1, 0.2380392156862745, 0.03224885844748859],
 [6, 0.020784313725490278, 0.029965753424657543],
 [5, 0.23529411764705888, 0.0019977168949771723],
 [1, 0.12358870967741936, 0.016680923866552594],
 [1, 0.12358870967741936, 0.0027088679783290603],
 [6, 0.08911290322580645, 0.0024237239806102207],
 [1, 0.315725806451613, 0.0819788993441688],
 [1, 0.6858870967741937, 0.020958083832335328],
 [1, 0.6858870967741937, 0.0069860279441117945],
 [6, 0.6514112903225807, 0.006700883946392955],
 [6, 0.0858870967741936, 0.002423723980610193],
 [1, 0.01370967741935486, 0.06387225548902192],
 [1, 0.38387096774193546, 0.0028514399771884524],
 [1, 0.1643145161290322, 0.07442258340461932],
 [1, 0.1643145161290322, 0.06045052751639579],
 [6, 0.12983870967741928, 0.06016538351867695],
 [5, 0.18286290322580648, 0.04405474764756201],
 [6, 0.059677419354838696, 0.0005702879954376794],
 [6, 0.45302419354838713, 0.08982035928143706],
 [6, 0.575, 0.08982035928143706],
 [6, 0.46733870967741925, 0.09637867122897048],
 [1, 0.759475806451613, 0.05332192757342458],
 [1, 0.6592741935483872, 0.021956087824351322],
 [1, 0.34529411764705875, 0.03257575757575759],
 [1, 0.3415686274509803, 0.0007575757575757347],
 [6, 0.08411764705882341, 0.0007575757575757347],
 [1, 0.34568627450980394, 0.09606060606060607],
 [1, 0.3419607843137255, 0.06424242424242421],
 [6, 0.0845098039215686, 0.06424242424242421],
 [4, 0.12588235294117645, 0.032727272727272716],
 [6, 0.08078431372549022, 0.032727272727272716],
 [6, 0.06882352941176473, 0.0009090909090908872],
 [4, 0.020392156862745092, 0.08863636363636368],
 [4, 0.16411764705882353, 0.05075757575757578],
 [1, 0.1635294117647059, 0.0010606060606060952],
 [1, 0.12141414141414142, 0.00014253135689851204],
 [1, 0.014141414141414232, 0.07112314709236031],
 [1, 0.1131313131313132, 0.09763397947548462],
 [6, 0.06646464646464656, 0.09734891676168758],
 [1, 0.20404040404040413, 0.04389965792474346],
 [3, 0.3717171717171717, 0.009122006841505104],
 [3, 0.40222222222222226, 0.018386545039908775],
 [1, 0.17595959595959587, 0.018386545039908775],
 [3, 0.38707070707070707, 0.0019954389965792796],
 [3, 0.3775757575757577, 0.03663055872291909],
 [1, 0.1513131313131313, 0.03663055872291909],
 [3, 0.3624242424242425, 0.020239452679589598],
 [1, 0.022589753933037415, 0.027080957810718342],
 [1, 0.07987091569181115, 0.027080957810718342],
 [1, 0.3275514320290439, 0.0002850627137970796],
 [1, 0.508874546187979, 0.09350057012542762],
 [1, 0.22725490196078435, 0.02100000000000002],
 [1, 0.22725490196078435, 0.0006666666666666765],
 [6, 0.17803921568627445, 0.0005000000000000004],
 [1, 0.22725490196078435, 0.041833333333333333],
 [1, 0.22725490196078435, 0.02149999999999999],
 [6, 0.17803921568627445, 0.021333333333333315],
 [4, 0.24, 0.0006666666666666488],
 [6, 0.20392156862745092, 0.0006666666666666488],
 [2, 0.28352941176470603, 0.028636363636363682],
 [1, 0.0659274193548387, 0.041904218928164205],
 [1, 0.3129988597491448, 0.0],
 [2, 0.11145952109464081, 0.024193548387096753],
 [2, 0.1160205245153934, 0.047983870967741926],
 [2, 0.8700114025085518, 0.0717741935483871],
 [5, 0.8614595210946407, 0.022177419354838745],
 [1, 0.10447548460661349, 0.00040322580645163475],
 [1, 0.28971774193548383, 0.0721414314228685],
 [1, 0.28971774193548383, 0.08982035928143711],
 [1, 0.05241935483870967, 0.01767892785856856],
 [6, 0.018951612903225792, 0.0172512118619903],
 [1, 0.010080645161290328, 0.03478756772169944],
 [1, 0.05221774193548388, 0.03464499572284002],
 [6, 0.018750000000000003, 0.03421727972626176],
 [1, 0.010282258064516145, 0.05203877958368974],
 [1, 0.05241935483870967, 0.05203877958368974],
 [6, 0.018951612903225792, 0.05161106358711148],
 [1, 0.010282258064516145, 0.06928999144568004],
 [1, 0.05241935483870967, 0.06928999144568004],
 [6, 0.018951612903225792, 0.06886227544910178],
 [1, 0.5510080645161292, 0.06871970345024236],
 [6, 0.5175403225806453, 0.0682919874536641],
 [1, 0.4114919354838711, 0.06871970345024236],
 [1, 0.010282258064516145, 0.08668377530652976],
 [1, 0.05262096774193546, 0.08654120330767034],
 [6, 0.019153225806451582, 0.08611348731109209],
 [6, 0.22914147521160833, 0.08836944127708102],
 [6, 0.22914147521160833, 0.07326111744583808],
 [6, 0.23155985489721898, 0.08580387685290763],
 [6, 0.2295445384925433, 0.09848916761687565],
 [1, 0.08523375142531359, 0.028427419354838723],
 [6, 0.1783067274800456, 0.006048387096774188],
 [1, 0.11730330672748006, 0.0058467741935483986],
 [6, 0.14523945267958951, 0.07903225806451614],
 [1, 0.08423603192702396, 0.07883064516129035],
 [6, 0.14523945267958951, 0.010887096774193528],
 [2, 0.09093500570125429, 0.011290322580645107],
 [1, 0.8661630558722919, 0.00040322580645157924],
 [1, 0.8661630558722919, 0.026814516129032295],
 [1, 0.8661630558722919, 0.00040322580645169026],
 [1, 0.16034777651083237, 0.0985887096774194],
 [1, 0.16049030786773089, 0.07217741935483879],
 [1, 0.05215686274509801, 0.08257575757575758],
 [1, 0.36078431372549014, 0.08181818181818182],
 [1, 0.14392156862745087, 0.018939393939393923],
 [1, 0.14392156862745087, 0.00045454545454545747],
 [6, 0.09666666666666668, 0.00045454545454545747],
 [1, 0.14803921568627443, 0.05590909090909088],
 [1, 0.14803921568627443, 0.037424242424242415],
 [6, 0.10078431372549024, 0.037424242424242415],
 [6, 0.11058823529411765, 0.00045454545454548523],
 [1, 0.13607843137254894, 0.09409090909090909],
 [1, 0.13607843137254894, 0.07560606060606062],
 [6, 0.08882352941176475, 0.07560606060606062],
 [6, 0.09862745098039216, 0.03863636363636369],
 [5, 0.13235294117647056, 0.018939393939393923],
 [3, 0.12960784313725493, 0.0003030303030303605],
 [5, 0.22274509803921566, 0.013484848484848433],
 [3, 0.22019607843137254, 0.0],
 [1, 0.11028225806451619, 0.05289421157684631],
 [1, 0.04939516129032262, 0.06715141146278872],
 [1, 0.16633064516129037, 0.05289421157684631],
 [1, 0.20141129032258065, 0.05118334759053322],
 [1, 0.13, 0.04696969696969697],
 [1, 0.11254901960784319, 0.017121212121212148],
 [1, 0.1072549019607843, 0.05681818181818177],
 [1, 0.28960784313725485, 0.0719696969696969],
 [6, 0.2488906817265027, 0.05216647662485746],
 [1, 0.6509471987101976, 0.09506841505131129],
 [1, 0.24304715840386948, 0.04774800456100345],
 [1, 0.6662636033857317, 0.015820980615735475],
 [4, 0.1850060459492141, 0.001140250855188124],
 [6, 0.1475211608222492, 0.0012827822120866639],
 [3, 0.16041918581217252, 0.06214367160775369],
 [3, 0.04373236598145913, 0.06371151653363749],
 [6, 0.08843137254901967, 0.028333333333333335],
 [1, 0.1286274509803922, 0.0],
 [1, 0.23356192012908422, 0.08685714285714285],
 [1, 0.5121016538926986, 0.052714285714285714],
 [4, 0.14703509479628885, 0.0],
 [6, 0.11879790237999188, 0.0],
 [4, 0.1385639370713998, 0.06157142857142858],
 [6, 0.11032674465510284, 0.06157142857142858],
 [1, 0.13876563130294473, 0.026857142857142857],
 [6, 0.08652682533279554, 0.026857142857142857],
 [1, 0.1304961678096006, 0.01314285714285715],
 [6, 0.10084711577248895, 0.01314285714285715],
 [1, 0.1379588543767647, 0.00114285714285714],
 [1, 0.2125857200484067, 0.043571428571428594],
 [1, 0.22851956434045984, 0.043571428571428594],
 [1, 0.021177894312222723, 0.0007142857142856673],
 [1, 0.5304558289632916, 0.047142857142857264],
 [1, 0.2644211375554659, 0.04551940639269407],
 [1, 0.22388059701492546, 0.012699771689497721],
 [6, 0.021379588543767536, 0.012699771689497721],
 [1, 0.8101571946795647, 0.06330196749358424],
 [1, 0.2202740830310358, 0.0005702879954376794],
 [1, 0.8099556630390972, 0.08582834331337325],
 [1, 0.22007255139056825, 0.02309666381522668],
 [4, 0.20858524788391786, 0.00042771599657825954],
 [1, 0.2370012091898427, 0.09053321927573421],
 [4, 0.2255139056831923, 0.06786427145708579],
 [1, 0.26239419588875446, 0.02266894781864842],
 [1, 0.14723678902783377, 0.04751712328767124],
 [1, 0.10830980233965304, 0.015125570776255703],
 [1, 0.19745865268253326, 0.09860159817351599],
 [6, 0.021782977006857607, 0.09860159817351599],
 [5, 0.19302137958854382, 0.06906392694063929],
 [6, 0.021782977006857607, 0.07248858447488588],
 [2, 0.14985881403791856, 0.04152397260273971],
 [6, 0.021782977006857607, 0.04152397260273971],
 [4, 0.027430415490117044, 0.013555936073059327],
 [6, 0.5, 0.03948118586088939],
 [1, 0.17861457913813933, 0.014680729760547323],
 [6, 0.014498590414820822, 0.015535917901938423],
 [1, 0.04973821989528793, 0.014965792474344375],
 [6, 0.05424259681093391, 0.012985705657338475],
 [1, 0.17653758542141218, 0.017716931749547027],
 [6, 0.015660592255125283, 0.024562109925508446],
 [6, 0.43379840546697035, 0.024562109925508446],
 [6, 0.015802961275626443, 0.04912421985101678],
 [6, 0.43379840546697035, 0.04912421985101678],
 [6, 0.013952164009111617, 0.07459230924099058],
 [6, 0.05068337129840547, 0.09915441916649892],
 [6, 0.43379840546697035, 0.09824843970203345],
 [6, 0.6775341685649203, 0.006744513791020812],
 [6, 0.04840546697038728, 0.05908999396013692],
 [1, 0.014320290439693406, 0.0950684150513113],
 [1, 0.10609116579265832, 0.0950684150513113],
 [3, 0.6306978620411456, 0.040621436716077486],
 [1, 0.24862745098039207, 0.015540347875677207],
 [6, 0.024313725490195948, 0.016110635871114914],
 [1, 0.05078431372549019, 0.058169375534645004],
 [5, 0.20492133924969747, 0.027968036529680364],
 [1, 0.16962484872932637, 0.0001426940639269389],
 [5, 0.22388059701492535, 0.05037100456621004],
 [1, 0.18858410649455426, 0.022545662100456616],
 [1, 0.0580879386849536, 0.02237742303306728],
 [6, 0.04094392900363042, 0.00441847206385404],
 [1, 0.03731343283582084, 0.07725199543899658],
 [6, 0.020169423154497657, 0.05929304446978334],
 [1, 0.012706736587333567, 0.04347206385404792],
 [6, 0.05313725490196086, 0.0001515151515151525],
 [6, 0.13549019607843138, 0.08651515151515152],
 [1, 0.5503921568627451, 0.00015151515151518025],
 [1, 0.30980392156862746, 0.052424242424242484],
 [1, 0.5876470588235295, 0.09500000000000003],
 [6, 0.15705882352941192, 0.09469696969696967],
 [1, 0.3664705882352941, 0.001818181818181941],
 [1, 0.030980392156862768, 0.04939393939393932],
 [1, 0.16801129487696653, 0.08105022831050229],
 [1, 0.6789027833803953, 0.08105022831050229],
 [1, 0.1643807987091569, 0.01840753424657532],
 [1, 0.35630794034663443, 0.04119156214367159],
 [6, 0.3176138653768642, 0.04119156214367159],
 [1, 0.3681983071342201, 0.018101482326111723],
 [1, 0.027206771463119728, 0.051453819840364845],
 [1, 0.03909713825070538, 0.02836374002280498],
 [1, 0.75534058847239, 0.09335803876852905],
 [6, 0.7166465135026199, 0.09335803876852905],
 [1, 0.3299072954453849, 0.0962086659064994],
 [1, 0.05824264409512292, 0.09791904218928163],
 [1, 0.7672309552599758, 0.07026795895096918],
 [3, 0.145304312777106, 0.0917901938426453],
 [1, 0.08862745098039226, 0.04530303030303032],
 [4, 0.09823529411764709, 0.018333333333333313],
 [1, 0.09411764705882353, 0.019545454545454533],
 [1, 0.24098039215686273, 0.0],
 [6, 0.053137254901960806, 0.0018181818181818299],
 [1, 0.04607843137254902, 0.07696969696969697],
 [1, 0.03941176470588237, 0.046060606060606066],
 [1, 0.2889110604332953, 0.0],
 [1, 0.10647092360319266, 0.07255139056831905],
 [6, 0.3550403225806452, 0.07682440136830104],
 [6, 0.33608870967741933, 0.03420752565564422],
 [6, 0.07396211205159209, 0.0004275940706955639],
 [6, 0.021765417170495738, 0.05116875712656782],
 [1, 0.16196078431372551, 0.053484848484848524],
 [6, 0.025816861637757094, 0.0],
 [6, 0.011899959661153647, 0.029789053591790182],
 [1, 0.6107301331181929, 0.0255131128848346],
 [1, 0.3864461476401775, 0.0255131128848346],
 [1, 0.627672448567971, 0.010547320410490335],
 [3, 0.16337232755143216, 0.009549600912200695],
 [4, 0.1772892295280356, 0.0],
 [6, 0.18467741935483883, 0.05927095292766932],
 [6, 0.3231854838709678, 0.08869115958668194],
 [6, 0.32641129032258065, 0.0752009184845005],
 [3, 0.4477822580645162, 0.01449483352468417],
 [3, 0.37177419354838703, 0.0],
 [6, 0.03427419354838701, 0.0030137772675085106],
 [6, 0.08124999999999993, 0.00014351320321459582],
 [1, 0.20078431372549013, 0.015757575757575776],
 [1, 0.28647058823529414, 0.09212121212121216],
 [6, 0.21921568627450982, 0.050000000000000044],
 [1, 0.36392156862745095, 0.09272727272727274],
 [2, 0.038235294117647034, 0.09272727272727274],
 [6, 0.2966666666666667, 0.05060606060606063],
 [1, 0.23538895606610238, 0.0005702879954377349],
 [6, 0.19609028617492946, 0.00042771599657831505],
 [1, 0.2349858927851673, 0.026660963786712316],
 [6, 0.19568722289399437, 0.026518391787852896],
 [4, 0.24425634824667475, 0.0004277159965782873],
 [6, 0.21402660217654174, 0.0004277159965782873],
 [1, 0.6787585650947199, 0.013116623895067014],
 [6, 0.6394598952035468, 0.012974051896207595],
 [3, 0.7996775493752519, 0.007984031936127733],
 [4, 0.13240628778718266, 0.0005702879954376794],
 [3, 0.011487303506650545, 0.06287425149700598],
 [1, 0.02398226521563887, 0.06643855146849154],
 [1, 0.02614511552492904, 0.03826384922901199],
 [1, 0.00020267531414674966, 0.012849800114220433],
 [1, 0.06850425618159717, 0.03826384922901199],
 [1, 0.042561815970814876, 0.012849800114220433],
 [1, 0.042359140656668126, 0.0],
 [1, 0.16984191325496556, 0.03869217589948601],
 [1, 0.14389947304418327, 0.01327812678469445],
 [1, 0.14369679773003652, 0.0004283266704740163],
 [1, 0.05026347790839081, 0.07652769845802398],
 [1, 0.024321037697608516, 0.051113649343232426],
 [1, 0.024118362383461767, 0.03826384922901199],
 [2, 0.03283340089177145, 0.012849800114220433],
 [2, 0.03242805026347795, 0.0],
 [1, 0.10490196078431373, 0.012727272727272726],
 [1, 0.10490196078431373, 0.0],
 [1, 0.21647058823529408, 0.0934848484848485],
 [1, 0.27313725490196084, 0.03590909090909092],
 [1, 0.17459677419354835, 0.012974051896207595],
 [1, 0.20584677419354835, 0.02609067579127461],
 [1, 0.20584677419354835, 0.011833475905332236],
 [6, 0.14032258064516123, 0.01183347590533218],
 [1, 0.027217741935483875, 0.0825491873396065],
 [1, 0.027217741935483875, 0.06829198745366413],
 [1, 0.25190396282431904, 0.03180034674696594],
 [1, 0.15715760939718604, 0.014280500045624561],
 [1, 0.15715760939718604, 0.0010493658180490695],
 [1, 0.2547437717826255, 0.06300757368373028],
 [1, 0.15999741835549253, 0.0454877269823889],
 [1, 0.15999741835549253, 0.03225659275481341],
 [1, 0.094746353427133, 0.03075098092891687],
 [1, 0.1335215812827753, 0.0012827822120866639],
 [6, 0.06938281565147239, 0.0011402508551881518],
 [1, 0.10669624848729327, 0.016248574686430983],
 [6, 0.04255748285599034, 0.01610604332953247],
 [1, 0.10669624848729327, 0.0002850627137970241],
 [1, 0.20673658733360223, 0.016248574686430983],
 [6, 0.14259782170229932, 0.01610604332953247],
 [1, 0.20673658733360223, 0.0002850627137970241],
 [1, 0.02686274509803921, 0.040606060606060645],
 [1, 0.21098039215686265, 0.00121212121212122],
 [1, 0.0472549019607843, 0.07681818181818167],
 [1, 0.03294117647058825, 0.06136363636363629],
 [6, 0.15627450980392157, 0.0465151515151514],
 [1, 0.10999999999999999, 0.045454545454545414],
 [1, 0.03333333333333338, 0.0465151515151514],
 [1, 0.1723529411764706, 0.0310606060606059],
 [1, 0.34852887758808565, 0.030670103092783507],
 [3, 0.20395931710860882, 0.006314432989690699],
 [1, 0.3159215328467153, 0.039090909090909065],
 [1, 0.32892335766423353, 0.05348484848484847],
 [1, 0.1719890510948905, 0.012575757575757573],
 [1, 0.3163777372262774, 0.0962121212121212],
 [1, 0.15944343065693434, 0.0553030303030303],
 [1, 0.15921532846715336, 0.040909090909090895],
 [6, 0.1261405109489051, 0.040909090909090895],
 [1, 0.1731295620437956, 0.08303030303030298],
 [1, 0.17290145985401462, 0.06863636363636358],
 [6, 0.13982664233576636, 0.06863636363636358],
 [1, 0.17381386861313863, 0.09681818181818183],
 [1, 0.17358576642335766, 0.08242424242424243],
 [6, 0.1405109489051094, 0.08242424242424243],
 [6, 0.11541970802919699, 0.012121212121212144],
 [3, 0.43453467153284675, 0.043333333333333335],
 [6, 0.4468521897810219, 0.0009090909090908594],
 [3, 0.5896441605839415, 0.0613636363636364],
 [6, 0.6019616788321168, 0.018939393939393923],
 [3, 0.4281478102189781, 0.06909090909090909],
 [6, 0.44046532846715325, 0.026666666666666616],
 [3, 0.0812043795620438, 0.07833333333333337],
 [6, 0.09352189781021898, 0.03590909090909089],
 [3, 0.07504562043795618, 0.09257575757575759],
 [6, 0.08736313868613137, 0.050151515151515114],
 [6, 0.3934762773722627, 0.06984848484848483],
 [6, 0.4400091240875912, 0.07696969696969691],
 [6, 0.08713503649635036, 0.08636363636363631],
 [6, 0.6028740875912408, 0.08378787878787874],
 [6, 0.4413777372262774, 0.09136363636363631],
 [1, 0.3483120437956204, 0.09030303030303033],
 [1, 0.6192974452554745, 0.08984848484848484],
 [6, 0.18784313725490198, 0.01787878787878794],
 [6, 0.24392156862745096, 0.018030303030303063],
 [6, 0.24392156862745096, 0.0],
 [6, 0.1707843137254903, 0.035909090909090946],
 [3, 0.6003921568627452, 0.016818181818181788],
 [6, 0.1707843137254903, 0.017878787878787883],
 [6, 0.016470588235294112, 0.014242424242424223],
 [3, 0.4611764705882353, 0.09121212121212119],
 [6, 0.031568627450980435, 0.09227272727272728],
 [6, 0.522156862745098, 0.028030303030303017],
 [6, 0.016470588235294112, 0.041666666666666685],
 [6, 0.1052941176470588, 0.044545454545454555],
 [6, 0.017058823529411765, 0.09560606060606064],
 [6, 0.36098039215686284, 0.0995454545454546],
 [3, 0.7905882352941177, 0.08045454545454545],
 [6, 0.36098039215686284, 0.08151515151515154],
 [6, 0.8515686274509804, 0.017272727272727273],
 [1, 0.19921568627450992, 0.06712121212121214],
 [1, 0.18862745098039224, 0.022575757575757582],
 [1, 0.18862745098039224, 0.00045454545454545747],
 [6, 0.14137254901960783, 0.00045454545454545747],
 [2, 0.2623529411764705, 0.06954545454545458],
 [2, 0.4319607843137255, 0.06196969696969701],
 [3, 0.18529411764705883, 0.011818181818181728],
 [1, 0.08058823529411765, 0.012878787878787712],
 [4, 0.026666666666666672, 0.012878787878787712],
 [3, 0.34352941176470586, 0.011818181818181728],
 [1, 0.23882352941176468, 0.012878787878787712],
 [4, 0.1849019607843137, 0.012878787878787712],
 [1, 0.0896078431372549, 0.011818181818181728],
 [1, 0.7086828859060403, 0.0006060606060606377],
 [1, 0.7040687919463087, 0.08303030303030304],
 [1, 0.07843959731543626, 0.0821212121212121],
 [1, 0.6795302013422819, 0.0934848484848485],
 [1, 0.053901006711409405, 0.09257575757575756],
 [6, 0.1088506711409396, 0.010151515151515161],
 [6, 0.3794043624161073, 0.030757575757575706],
 [6, 0.3022231543624161, 0.03636363636363632],
 [6, 0.3238255033557047, 0.03636363636363632],
 [6, 0.4305788590604027, 0.03575757575757571],
 [6, 0.45218120805369133, 0.03575757575757571],
 [6, 0.3022231543624161, 0.08454545454545453],
 [6, 0.3236157718120805, 0.08454545454545453],
 [6, 0.43015939597315433, 0.0837878787878788],
 [6, 0.45176174496644295, 0.0837878787878788],
 [1, 0.6256291946308725, 0.0009090909090909427],
 [6, 0.2036493288590604, 0.00121212121212122],
 [6, 0.027474832214765044, 0.039999999999999925],
 [6, 0.06375838926174493, 0.05560606060606066],
 [6, 0.15331375838926165, 0.07045454545454544],
 [6, 0.19651845637583887, 0.07439393939393935],
 [6, 0.4183138362242839, 0.059931506849315086],
 [6, 0.5865268253327955, 0.07320205479452055],
 [1, 0.2045179507866074, 0.012842465753424681],
 [6, 0.018152480839047858, 0.012842465753424681],
 [3, 0.03146430012101642, 0.007420091324200906],
 [3, 0.11355385235982252, 0.09817351598173524],
 [4, 0.14360629286002424, 0.06720890410958913],
 [6, 0.03348124243646633, 0.06720890410958913],
 [5, 0.4945542557482856, 0.00042808219178080975],
 [3, 0.2466720451795078, 0.09731735159817362],
 [6, 0.032472771278741486, 0.09831621004566216],
 [3, 0.1236385639370714, 0.09018264840182655],
 [4, 0.013110125050423416, 0.09118150684931514],
 [4, 0.27672448567970953, 0.06635273972602751],
 [6, 0.16659943525615162, 0.06635273972602751],
 [3, 0.06171843485276318, 0.06521118721461194],
 [4, 0.1393707139975796, 0.07919520547945214],
 [6, 0.029245663574021696, 0.07919520547945214],
 [5, 0.49031867688584096, 0.012414383561643816],
 [5, 0.45018152480839047, 0.03652968036529669],
 [5, 0.4497781363453005, 0.048373287671232834],
 [1, 0.03960784313725489, 0.050757575757575724],
 [1, 0.2405882352941177, 0.09833333333333333],
 [6, 0.02450980392156865, 0.09833333333333333],
 [6, 0.1698039215686275, 0.07424242424242428],
 [6, 0.05078431372549019, 0.0],
 [1, 0.28274509803921566, 0.028484848484848488],
 [6, 0.019019607843137276, 0.015303030303030304],
 [1, 0.19490196078431388, 0.015303030303030304],
 [1, 0.4354901960784314, 0.008787878787878789],
 [1, 0.15372549019607845, 0.016060606060606053],
 [6, 0.003137254901960776, 0.049393939393939434],
 [1, 0.796448087431694, 0.06718750000000001],
 [6, 0.1407103825136612, 0.0678125],
 [1, 0.8515482695810564, 0.06718750000000001],
 [6, 0.1958105646630236, 0.0678125],
 [1, 0.819672131147541, 0.08374999999999999],
 [6, 0.16393442622950816, 0.08437499999999998],
 [6, 0.020947176684881594, 0.016562500000000036],
 [6, 0.7595628415300545, 0.012187500000000018],
 [6, 0.020947176684881594, 0.03187500000000004],
 [1, 0.37112932604735893, 0.028437499999999893],
 [1, 0.0692156862745098, 0.0009090909090909149],
 [1, 0.1347058823529412, 0.09924242424242424],
 [1, 0.24156862745098043, 0.09924242424242424],
 [1, 0.22882352941176473, 0.0],
 [6, 0.06794354838709682, 0.034492588369441246],
 [6, 0.0473790322580645, 0.03477765108323838],
 [1, 0.2798039215686275, 0.0],
 [1, 0.1164705882352941, 0.04181818181818181],
 [1, 0.07156862745098036, 0.016515151515151594],
 [1, 0.30098039215686273, 0.015606060606060623],
 [1, 0.45274509803921564, 0.016515151515151594],
 [1, 0.5068627450980392, 0.02939393939393939],
 [1, 0.18725490196078431, 0.0],
 [1, 0.1503921568627451, 0.0013636363636363447],
 [1, 0.22372549019607846, 0.08121212121212121],
 [1, 0.2833333333333333, 0.01575757575757572],
 [6, 0.023725490196078503, 0.01575757575757572],
 [6, 0.07294117647058823, 0.016515151515151538],
 [3, 0.03019607843137251, 0.09136363636363637],
 [1, 0.06716417910447761, 0.036672374429223775],
 [1, 0.23117647058823532, 0.03515151515151515],
 [1, 0.26078431372549016, 0.05242424242424242],
 [1, 0.1049019607843138, 0.016212121212121212],
 [3, 0.843529411764706, 0.025000000000000022],
 [3, 0.5682352941176471, 0.06742424242424244],
 [1, 0.35058823529411764, 0.04424242424242425],
 [1, 0.1743137254901962, 0.095],
 [6, 0.12980392156862763, 0.09515151515151515],
 [1, 0.2280241935483871, 0.018106643855146848],
 [6, 0.015120967741935498, 0.018106643855146848],
 [1, 0.2280241935483871, 0.04833190761334474],
 [6, 0.015120967741935498, 0.04833190761334474],
 [4, 0.021774193548387055, 0.018391787852865715],
 [1, 0.029032258064516148, 0.0718562874251497],
 [4, 0.00745967741935484, 0.01283147989734823],
 [1, 0.23518742442563478, 0.0005702879954377071],
 [6, 0.19568722289399432, 0.0004277159965782873],
 [1, 0.23518742442563478, 0.02666096378671229],
 [6, 0.19568722289399432, 0.02651839178785287],
 [3, 0.22934300685207576, 0.012688907898488755],
 [4, 0.1872228939943571, 0.00042771599657825954],
 [6, 0.15699314792422409, 0.00042771599657825954],
 [1, 0.6773478436114471, 0.03963501568291988],
 [6, 0.6378476420798065, 0.03949244368406046],
 [3, 0.671503426037888, 0.02566295979469635],
 [3, 0.6717049576783556, 0.012546335899629335],
 [4, 0.6293833131801694, 0.013401767892785854],
 [6, 0.5991535671100363, 0.013401767892785854],
 [6, 0.18158000806126573, 0.00042771599657825954],
 [1, 0.16082224909310763, 0.04690618762475049],
 [6, 0.09633212414349057, 0.08996293128029653],
 [1, 0.6027811366384522, 0.04647847162817223],
 [1, 0.07234985892785167, 0.09965782720273739],
 [1, 0.3307843137254901, 0.029999999999999995],
 [1, 0.025882352941176467, 0.043939393939393945],
 [1, 0.17098039215686278, 0.08545454545454548],
 [5, 0.14607843137254906, 0.024393939393939412],
 [1, 0.040980392156862666, 0.02909090909090908],
 [1, 0.030067758328627903, 0.018445322793148877],
 [1, 0.0966967814793902, 0.018539431582909844],
 [1, 0.07552230378317337, 0.0808394504046678],
 [1, 0.0522303783173349, 0.05815923207227555],
 [6, 0.036420101637492985, 0.012516469038208189],
 [1, 0.429983060417843, 0.05251270468661762],
 [1, 0.4690852625635234, 0.05251270468661784],
 [1, 0.4151609260304912, 0.07538114059853196],
 [1, 0.27647058823529413, 0.0393939393939394],
 [1, 0.7564705882352941, 0.0028787878787878696],
 [1, 0.3407843137254901, 0.03848484848484848],
 [1, 0.8207843137254901, 0.0019696969696969546],
 [1, 0.3831372549019608, 0.038636363636363635],
 [1, 0.8631372549019608, 0.002121212121212107],
 [1, 0.12137254901960785, 0.020454545454545447],
 [1, 0.3990196078431373, 0.07590909090909091],
 [1, 0.8790196078431373, 0.039393939393939384],
 [2, 0.06411764705882353, 0.07878787878787882],
 [3, 0.06803921568627452, 0.024393939393939412],
 [3, 0.17941176470588235, 0.09454545454545471],
 [1, 0.4257765227914481, 0.0178164196123147],
 [6, 0.054659136748689, 0.09777651083238315],
 [4, 0.057684550221863654, 0.07924743443557586],
 [6, 0.015328761597418306, 0.07924743443557586],
 [6, 0.43041549011698255, 0.09692132269099202],
 [4, 0.43344090359015724, 0.07839224629418473],
 [6, 0.39108511496571186, 0.07839224629418473],
 [2, 0.3442920532472771, 0.06014823261117444],
 [6, 0.4953610326744655, 0.09777651083238315],
 [4, 0.4983864461476402, 0.07924743443557586],
 [6, 0.4560306575231948, 0.07924743443557586],
 [2, 0.40923759580476005, 0.06100342075256557],
 [1, 0.16470588235294126, 0.014090909090909098],
 [6, 0.3688235294117647, 0.04181818181818181],
 [1, 0.5683743444937475, 0.012257696693272507],
 [1, 0.24929463925836354, 0.014970059880239528],
 [1, 0.24929463925836354, 0.00014257199885941985],
 [6, 0.20898831116485286, 0.0],
 [1, 0.24929463925836354, 0.04448246364414027],
 [1, 0.24929463925836354, 0.02965497576276016],
 [6, 0.20898831116485286, 0.02951240376390074],
 [4, 0.258565094719871, 0.00014257199885941985],
 [6, 0.22752922208786777, 0.00014257199885941985],
 [4, 0.09028617492946392, 0.0798403193612775],
 [6, 0.0592503022974607, 0.0798403193612775],
 [2, 0.023176138653768635, 0.06529797547761623],
 [1, 0.09250302297460701, 0.04106073567151414],
 [1, 0.5562273276904475, 0.09566581123467349],
 [6, 0.5159209995969367, 0.09552323923581407],
 [4, 0.5654977831519549, 0.06615340747077275],
 [6, 0.5344619105199516, 0.06615340747077275],
 [2, 0.4983877468762596, 0.05161106358711148],
 [2, 0.32184602982668276, 0.051325919589392616],
 [1, 0.567714631197098, 0.027373823781009388],
 [3, 0.03950020153164048, 0.00584545195323638],
 [3, 0.6320032245062475, 0.026946107784431073],
 [6, 0.24778225806451615, 0.047359357060849594],
 [6, 0.32641129032258065, 0.0898392652123996],
 [1, 0.05970149253731338, 0.03534777651083238],
 [1, 0.06070996369503834, 0.07668187001140242],
 [1, 0.014723678902783366, 0.0870866590649943],
 [1, 0.15933844292053245, 0.09763397947548463],
 [1, 0.056474384832593705, 0.045519406392694084],
 [1, 0.25514320290439696, 0.045519406392694084],
 [1, 0.19342476805163378, 0.01155821917808221],
 [6, 0.02561516740621217, 0.01155821917808221],
 [1, 0.35529411764705887, 0.021818181818181813],
 [1, 0.1874509803921569, 0.0013636363636363585],
 [1, 0.3666666666666667, 0.035909090909090925],
 [1, 0.19882352941176473, 0.01545454545454547],
 [1, 0.1976470588235294, 0.0009090909090909149],
 [1, 0.34725490196078435, 0.0009090909090909149],
 [1, 0.13755546591367485, 0.01881413911060434],
 [6, 0.135941912061315, 0.0],
 [6, 0.021782977006857607, 0.08608893956670466],
 [6, 0.3499394917305364, 0.08608893956670466],
 [6, 0.06897942718838246, 0.0],
 [1, 0.15302419354838703, 0.05957810718358038],
 [1, 0.41572580645161283, 0.01795895096921321],
 [1, 0.4195564516129032, 0.0012827822120866361],
 [6, 0.36209677419354835, 0.001140250855188124],
 [1, 0.10117647058823526, 0.03212121212121212],
 [2, 0.17882352941176471, 0.04863636363636362],
 [1, 0.46980392156862744, 0.014545454545454528],
 [6, 0.013921568627450975, 0.01818181818181816],
 [2, 0.16647058823529415, 0.08196969696969697],
 [1, 0.45745098039215687, 0.04787878787878787],
 [6, 0.0015686274509804088, 0.0515151515151515],
 [6, 0.4319607843137254, 0.09999999999999998],
 [1, 0.5327450980392157, 0.07424242424242422],
 [1, 0.5042989417989419, 0.08531746031746029],
 [1, 0.1106150793650793, 0.05047123015873013],
 [1, 0.15062830687830675, 0.08395337301587301],
 [6, 0.007440476190476192, 0.02380952380952378],
 [6, 0.005456349206349215, 0.051711309523809534],
 [6, 0.004464285714285726, 0.06560019841269837],
 [6, 0.00347222222222221, 0.07948908730158732],
 [6, 0.17406212182331582, 0.0004285714285714448],
 [6, 0.15026220250100847, 0.035714285714285754],
 [6, 0.1698265429608713, 0.07500000000000001],
 [6, 0.25030254134731744, 0.07500000000000001],
 [2, 0.4756048387096775, 0.01767388825541616],
 [2, 0.4756048387096775, 0.0],
 [2, 0.05544354838709678, 0.07554161915621438],
 [2, 0.05544354838709678, 0.05786773090079822],
 [2, 0.4296370967741935, 0.09278791334093506],
 [2, 0.5921370967741936, 0.09250285062713792],
 [1, 0.11653225806451606, 0.09250285062713792],
 [2, 0.6836693548387097, 0.09250285062713792],
 [1, 0.2080645161290322, 0.09250285062713792],
 [1, 0.0429435483870968, 0.0746864310148232],
 [6, 0.09717741935483876, 0.00014253135689856755],
 [6, 0.010282258064516159, 0.009977194982896176],
 [3, 0.36995967741935487, 0.004133409350056905],
 [6, 0.09717741935483876, 0.030501710376282798],
 [3, 0.45685483870967747, 0.024657924743443527],
 [6, 0.09717741935483876, 0.0002850627137971351],
 [6, 0.010282258064516159, 0.04019384264538206],
 [3, 0.36995967741935487, 0.03435005701254279],
 [3, 0.36995967741935487, 0.009122006841505215],
 [6, 0.010282258064516159, 0.009977194982896398],
 [1, 0.15147236789027835, 0.059293044469783354],
 [1, 0.10367083501411861, 0.01795895096921324],
 [1, 0.10367083501411861, 0.00014253135689851204],
 [6, 0.060709963695038394, 0.00014253135689851204],
 [1, 0.400161355385236, 0.03591790193842648],
 [1, 0.05183541750705929, 0.05387685290763969],
 [1, 0.03388463089955629, 0.052879133409350076],
 [2, 0.49193223073820086, 0.03862599771949837],
 [1, 0.2888261395724082, 0.03862599771949837],
 [2, 0.12121823315853175, 0.03862599771949837],
 [2, 0.6377571601452199, 0.019241733181299958],
 [1, 0.4346510689794272, 0.019241733181299958],
 [2, 0.2670431625655507, 0.019241733181299958],
 [3, 0.032069382815651526, 0.01824401368301032],
 [1, 0.21549019607843134, 0.04439393939393939],
 [1, 0.2377974989915288, 0.02879133409350057],
 [1, 0.22125857200484067, 0.016248574686431018],
 [6, 0.021177894312222723, 0.017958950969213232]]

In [3]:
pca = PCA(n_components=1)

In [4]:
pca.fit_transform(X)

array([[-1.98686585],
       [-1.98640142],
       [ 3.01565126],
       [-1.98632411],
       [-1.99416836],
       [-1.98697343],
       [ 3.01562743],
       [-1.98480631],
       [-1.99083463],
       [-1.9836397 ],
       [-1.99083463],
       [-1.9836397 ],
       [-1.99418798],
       [-1.98699305],
       [ 3.01560781],
       [ 2.01264346],
       [ 3.01561617],
       [-1.98477076],
       [ 3.01512684],
       [ 3.01512684],
       [ 1.01448902],
       [-1.99416836],
       [-1.98697343],
       [ 3.01562743],
       [ 3.0156376 ],
       [-1.98685315],
       [-1.9934173 ],
       [-1.98537263],
       [-1.98536945],
       [ 3.01473334],
       [-1.98711748],
       [-1.98686414],
       [-1.99342829],
       [-1.98538362],
       [-1.98538044],
       [ 3.01472235],
       [-0.98503817],
       [-1.98622089],
       [-1.99278504],
       [-1.98474037],
       [-1.98473719],
       [ 3.01536559],
       [-0.98439492],
       [-1.99146842],
       [-1.99323955],
       [-1

In [5]:
dataDir = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data")
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data")


In [15]:
def getNeighbourInfo(candidates, data):
    vocab = ["invoice", "inv", "receipt", "bill", "order", "payment", "due", "delivery", "date", "dated"]
    emb = {
        "invoice" : 1,
        "inv" : 1,
        "bill" : 1,
        "receipt" : 1,
        "order" : 2,
        "payment" : 3,
        "due" : 4,
        "delivery" : 5,
        "date" : 6,
        "dated" : 6
    }
    for df in candidates:
        for col in ["invoice", "order", "payment", "due", "delivery", "date"]:
            df[col] = 0
    for df in candidates:
        result = []
        for index in df.index:
            Neighbours = {}
            X = df["x1"][index] + df["x2"][index]
            Y = df["y1"][index] + df["y2"][index]
            X/=2
            Y/=2
            for idx in data.index:
                text = data["text"][idx].lower()
                X2 = data["X"][idx]
                Y2 = data["Y"][idx]
                if Y-Y2 >= 0 and Y-Y2 <= 0.1  and X2<X and text in emb:
                    N = []
                    N.append(emb[text])
                    N.append(X-X2)
                    N.append(Y-Y2)
                    enc = pca.transform([N])
                    Neighbours[data["text"][idx].lower()] = enc[0]
            colDict = {
                "invoice" : "invoice",
                "inv" : "invoice",
                "bill" : "invoice",
                "receipt" : "invoice",
                "date" : "date",
                "dated" : "date",
                "order" : "order",
                "payment" : "payment",
                "due" : "due",
                "delivery" : "delivery",
            }
            for key in Neighbours:
                col = colDict[key]
                df[col][index] = Neighbours[key]

In [20]:
def createCandidates(file):

    data, candidateInfo = get_candidates(file)

    data = data[["text", "X", "Y"]]


    getNeighbourInfo(candidateInfo, data)

    return candidateInfo[1]

    # result = pd.concat(candidateInfo)

    # result.to_csv("Candidates/" + file.strip("/")[1])
    



In [21]:
createCandidates("Images/1.jpeg")

/tmp/ipykernel_18806/1025692589.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][index] = Neighbours[key]
/tmp/ipykernel_18806/1025692589.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][index] = Neighbours[key]
/tmp/ipykernel_18806/1025692589.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][index] = Neighbours[key]
/tmp/ipykernel_18806/1025692589.py:51: SettingWithCopyWarning: 
A value is trying to 

,text,x1,y1,x2,y2,Class,invoice,order,payment,due,delivery,date
0,Bhavani,0.130698,0.081050,0.188786,0.089897,invoice_date,0.000000,0,0,0,0,0.000000
1,21-07-2022,0.692618,0.073345,0.772489,0.080765,invoice_date,-1.986842,0,0,0,0,3.015555
2,"24,",0.300121,0.197203,0.323114,0.208333,invoice_date,0.000000,0,0,0,0,0.000000
3,1250,0.590561,0.352740,0.626059,0.361301,invoice_date,0.000000,0,0,0,0,0.000000
4,9 CGST 9%,0.274708,0.447774,0.649455,0.475457,invoice_date,0.000000,0,0,0,0,0.000000
5,"32,812.00 31,250.00",0.350545,0.618721,0.901573,0.638699,invoice_date,0.000000,0,0,0,0,0.000000
6,"32,812.00 Company's",0.061718,0.643550,0.901573,0.684075,invoice_date,0.000000,0,0,0,0,0.000000
7,1168120020000193,0.127874,0.716895,0.288826,0.726027,invoice_date,0.000000,0,0,0,0,0.000000
8,Bhavani,0.603873,0.859018,0.672449,0.872146,invoice_date,0.000000,0,0,0,0,0.000000


In [ ]:
R[1]

[{},
 {'date': array([[3.00983203]])},
 {'date': array([[3.00740798]])},
 {'invoice': array([[-1.98656314]]), 'dated': array([[3.01560152]])},
 {'payment': array([[0.01572183]])},
 {},
 {'payment': array([[0.01452295]]),
  'due': array([[1.01399254]]),
  'date': array([[3.01537058]]),
  'delivery': array([[2.00884938]])},
 {'payment': array([[0.01527539]]),
  'date': array([[3.01345442]]),
  'due': array([[1.01207637]])},
 {'due': array([[1.01405125]]),
  'date': array([[3.01542929]]),
  'delivery': array([[2.00890809]])},
 {},
 {'delivery': array([[2.00948133]])},
 {'delivery': array([[2.0094849]])},
 {},
 {},
 {},
 {}]

In [22]:
pwd

"/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data"

In [29]:
imagesDir = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")

In [30]:
for file in imagesDir:
    df = createCandidates(file)
    os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceDateDataset")
    df.to_csv(file + ".csv")
    os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")

/tmp/ipykernel_18806/1025692589.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][index] = Neighbours[key]
/tmp/ipykernel_18806/1025692589.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][index] = Neighbours[key]
/tmp/ipykernel_18806/1025692589.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][index] = Neighbours[key]
/tmp/ipykernel_18806/1025692589.py:51: SettingWithCopyWarning: 
A value is trying to 